# Data Engineering course

### Assignment 4

Student: D'Ambrosi Denis

19th November 2023

---

### Exercise 1: creating the DWH

##### Connecting to the backend

To actually run the code, the following requirements are needed:

1. An active PostgreSQL database and relative credentials
2. ```psycopg2-binary``` for connetting to the database from the Python environment
3. ```pandas``` for handling SQL query responses and computing the pivot table

> This notebook was run on Python 3.11.6 with psycopg2-binary 2.9.9 and Pandas 2.0.1

In [1]:
from psycopg2 import connect

# Please substitute the placeholders with your own credentials before running
db_credentials = {
    'host': '#####',
    'database': '#####',
    'user': '#####',
    'password': '#####',
}

conn = connect(**db_credentials)
cursor = conn.cursor()

conn.commit()

##### Creating the time table

> Note: in this section, we will substitute missing temporal informations (semester/month/day) with null values 

In [2]:
import os

RESULTS_PATH = './results/'
result_files = [ os.path.join(RESULTS_PATH, path)
                for path in os.listdir(RESULTS_PATH)
                if os.path.isfile(os.path.join(RESULTS_PATH, path))]
result_files

['./results/resultlist_parallel.json',
 './results/resultlist_datenbaken.json',
 './results/resultlist_process_engineering.json',
 './results/resultlist_data_engineering.json',
 './results/resultlist_interop.json',
 './results/resultlist_webtech.json',
 './results/resultlist_blockchains_bpm.json',
 './results/resultlist_esop.json']

In [3]:
import json

raw_grades = []
for file in result_files :
    with open(file, 'r') as fp :
        content = json.load(fp)
    raw_grades += [ (
        content['course'],
        content['examinator'],
        content['date'],
        *tuple(content['results'][i].values())
        ) for i in range(len(content['results'])) ]
raw_grades

[('623.502',
  '2077308266',
  '2022-02-02',
  '9000023',
  'Sofia Hofmann',
  '5',
  '922'),
 ('623.502',
  '2077308266',
  '2022-02-02',
  '9000026',
  'Neele Schmitt',
  '2',
  '911'),
 ('623.502',
  '2077308266',
  '2022-02-02',
  '9000027',
  'Johanna Werner',
  '3',
  '922'),
 ('623.502',
  '2077308266',
  '2022-02-02',
  '9000030',
  'Amelie Meier',
  '2',
  '911'),
 ('623.502',
  '2077308266',
  '2022-02-02',
  '9000031',
  'Amely Lehmann',
  '3',
  '922'),
 ('623.502', '2077308266', '2022-02-02', '9000034', 'Lukas Maier', '3', '911'),
 ('623.502',
  '2077308266',
  '2022-02-02',
  '9000035',
  'Jonas Köhler',
  '5',
  '922'),
 ('623.502', '2077308266', '2022-02-02', '9000038', 'Paul Walter', '5', '911'),
 ('623.502', '2077308266', '2022-02-02', '9000039', 'Felix Mayer', '1', '922'),
 ('623.502', '2077308266', '2022-02-02', '9000042', 'Fynn Fuchs', '3', '911'),
 ('623.502', '2077308266', '2022-02-02', '9000043', 'Tim Peters', '1', '922'),
 ('623.502', '2077308266', '2022-02-02'

In [4]:
times_raw = list(set([ tuple(map(int, entry[2].split('-'))) for entry in raw_grades ]))
times_raw

[(2022, 2, 3),
 (2022, 1, 25),
 (2022, 2, 2),
 (2022, 6, 24),
 (2022,),
 (2022, 2, 1)]

In [5]:
times = []
timeId = 0
for time in times_raw :
    if len(time) == 1 :
        new_entry = (timeId, time[0], None, None, None)
    else :
        semester = 'summer' if time[1] >= 3 and time[1] <= 9 else 'winter'
        day = time[2] if len(time) > 2 else None
        new_entry = (timeId, time[0], semester, time[1], day)
    times.append(new_entry)
    timeId += 1
times

[(0, 2022, 'winter', 2, 3),
 (1, 2022, 'winter', 1, 25),
 (2, 2022, 'winter', 2, 2),
 (3, 2022, 'summer', 6, 24),
 (4, 2022, None, None, None),
 (5, 2022, 'winter', 2, 1)]

In [6]:
conn.rollback()

In [7]:
create_table_query = f"""
    CREATE TABLE TIME (
        Id INT PRIMARY KEY,
        Year INT NOT NULL,
        Semester VARCHAR(6),
        Month INT,
        Day INT
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [8]:
insert_query = """
    INSERT INTO TIME (Id, Year, Semester, Month, Day)
    VALUES (%s, %s, %s, %s, %s);
"""

cursor.executemany(insert_query, times)
conn.commit()

##### Creating the student table

In [9]:
students = list(set([ entry[3:5] for entry in raw_grades ]))
students

[('9000059', 'Jannik Beck'),
 ('9000064', 'Leoni Schuster'),
 ('9000083', 'Sophia Schreiber'),
 ('9000063', 'Hanna Albrecht'),
 ('9000025', 'Nele Lange'),
 ('9000011', 'Emilie Schäfer'),
 ('9000082', 'Sofie Haas'),
 ('9000084', 'Sofia Graf'),
 ('9000098', 'Luka Arnold'),
 ('9000040', 'Maximilian Huber'),
 ('9000037', 'Luka König'),
 ('9000017', 'Sara Schröder'),
 ('9000014', 'Lilly Richter'),
 ('9000074', 'Lilli Stein'),
 ('9000045', 'Ben Scholz'),
 ('9000088', 'Johanna Kuhn'),
 ('9000044', 'Timm Lang'),
 ('9000052', 'Niklas Friedrich'),
 ('9000048', 'Max Jung'),
 ('9000049', 'Julian Hahn'),
 ('9000080', 'Laura Heinrich'),
 ('9000071', 'Emily Schumacher'),
 ('9000018', 'Lara Neumann'),
 ('9000027', 'Johanna Werner'),
 ('9000087', 'Neele Ziegler'),
 ('9000039', 'Felix Mayer'),
 ('9000003', 'Leoni Schneider'),
 ('9000028', 'Maja Schmitz'),
 ('9000046', 'Luis Möller'),
 ('9000036', 'Luca Herrmann'),
 ('9000053', 'Noah Keller'),
 ('9000020', 'Sophie Zimmermann'),
 ('9000022', 'Sophia Krüge

In [10]:
create_table_query = """
    CREATE TABLE STUDENT (
        Id CHAR(7) PRIMARY KEY,
        Name VARCHAR(30)
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [11]:
insert_query = """
    INSERT INTO STUDENT (Id, Name)
    VALUES (%s, %s);
"""

cursor.executemany(insert_query, students)
conn.commit()

##### Creating the fact table

In [12]:
def toTimeId (date : str) -> int :
    int_date = list(map(int, date.split('-')))
    int_date += [ None for _ in range(3-len(int_date)) ]
    id = list(filter(lambda entry : entry[1] == int_date[0],
                    filter(lambda entry : entry[3] == int_date[1],
                            filter(lambda entry : entry[4] == int_date[2], times))))
    assert not (len(id) < 1), f'Date ({date}) not found in the database'
    assert not (len(id) > 1), f'Ambiguous reference to the date ({date})'
    return id[0][0]

toTimeId( '2022-02-02' )

2

In [13]:
raw_grades[0]

('623.502', '2077308266', '2022-02-02', '9000023', 'Sofia Hofmann', '5', '922')

In [14]:
grades = [ (
    i,                      # Grade ID
    int(entry[5]),          # Grade
    entry[1],               # Lecturer ID
    entry[0],               # Course ID
    toTimeId(entry[2]),     # Time ID
    entry[3],               # Student ID
    int(entry[6]),           # StudyPlan ID
    ) for i, entry in enumerate(raw_grades) ]
grades

[(0, 5, '2077308266', '623.502', 2, '9000023', 922),
 (1, 2, '2077308266', '623.502', 2, '9000026', 911),
 (2, 3, '2077308266', '623.502', 2, '9000027', 922),
 (3, 2, '2077308266', '623.502', 2, '9000030', 911),
 (4, 3, '2077308266', '623.502', 2, '9000031', 922),
 (5, 3, '2077308266', '623.502', 2, '9000034', 911),
 (6, 5, '2077308266', '623.502', 2, '9000035', 922),
 (7, 5, '2077308266', '623.502', 2, '9000038', 911),
 (8, 1, '2077308266', '623.502', 2, '9000039', 922),
 (9, 3, '2077308266', '623.502', 2, '9000042', 911),
 (10, 1, '2077308266', '623.502', 2, '9000043', 922),
 (11, 2, '2077308266', '623.502', 2, '9000046', 911),
 (12, 4, '2077308266', '623.502', 2, '9000047', 922),
 (13, 3, '2077308266', '623.502', 2, '9000050', 911),
 (14, 5, '2077308266', '623.502', 2, '9000051', 922),
 (15, 1, '2077308266', '623.502', 2, '9000054', 911),
 (16, 1, '2077308266', '623.502', 2, '9000055', 922),
 (17, 2, '2077308266', '623.502', 2, '9000058', 911),
 (18, 4, '2077308266', '623.502', 2, '

In [15]:
create_table_query = """
    CREATE TABLE GRADE (
        Id INT PRIMARY KEY,
        Grade INT NOT NULL,
        LecturerId VARCHAR(10) NOT NULL,
        CourseId CHAR(7) NOT NULL,
        Date INT NOT NULL,
        StudentId CHAR(7) NOT NULL,
        StudyPlanId INT NOT NULL
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [16]:
insert_query = """
    INSERT INTO GRADE (Id, Grade, LecturerId, CourseId, Date, StudentId, StudyPlanId)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

cursor.executemany(insert_query, grades)
conn.commit()

##### Creating the lecturer table

In [17]:
AAU_METADATA_PATH = './aau_metadata.json'
with open(AAU_METADATA_PATH, 'r') as fp :
    content = json.load(fp)
content

{'id': 'ATEOS1000019137',
 'name': 'Alpen-Adria-Universität Klagenfurt',
 'state': 'Carinthia',
 'city': 'Klagenfurt am Wörthersee',
 'street': 'Universitätsstraße 65-67',
 'zip': '9020',
 'bachelor_study_plans': [{'id': '511',
   'name': 'Angewandte Informatik',
   'branch': 'Technical Studies'},
  {'id': '522', 'name': 'Wirtschaftsinformatik', 'branch': 'Economoics'}],
 'master_study_plans': [{'id': '922',
   'name': 'Wirtschaftsinformatik',
   'type': 'Master',
   'branch': 'Economics'},
  {'id': '911',
   'name': 'Angewandte Informatik',
   'type': 'Master',
   'branch': 'Technical Studies'}],
 'lecturers': [{'id': '36866626',
   'name': 'Assoc.Prof. Dipl.-Ing. Dr. Klaus Schöffmann',
   'department': 'ITEC'},
  {'id': '2077308266',
   'name': 'Postdoc-Ass. Dr. Dragi Kimovski',
   'department': 'ITEC'},
  {'id': '834107405',
   'name': 'O.Univ.-Prof. Dipl.-Ing. Dr. Johann Eder ',
   'department': 'ISYS'},
  {'id': '772243224',
   'name': 'Assoc.-Prof. DI. Dr. Julius Köpke',
   'depa

In [18]:
university_id = content['id']
university_name = content['name']

In [19]:
lecturers_raw = list(map(lambda entry : tuple(entry.values()), content['lecturers']))
lecturers_raw

[('36866626', 'Assoc.Prof. Dipl.-Ing. Dr. Klaus Schöffmann', 'ITEC'),
 ('2077308266', 'Postdoc-Ass. Dr. Dragi Kimovski', 'ITEC'),
 ('834107405', 'O.Univ.-Prof. Dipl.-Ing. Dr. Johann Eder ', 'ISYS'),
 ('772243224', 'Assoc.-Prof. DI. Dr. Julius Köpke', 'ISYS')]

In [20]:
def process_lecturer_information (lecturer : str) -> tuple[str,str,str] :
    lecturer_as_list = lecturer.strip().split(' ')
    name = ' '.join(lecturer_as_list[-2:])
    rank = ''
    if 'assoc' in lecturer_as_list[0].lower() : rank = 'Ass Prof'
    if 'post' in lecturer_as_list[0].lower() : rank = 'Postdoc-Ass'
    if 'univ' in lecturer_as_list[0].lower() : rank = 'Univ Ass'
    if 'o.univ' in lecturer_as_list[0].lower() : rank = 'Prof'
    title = ''
    if 'di' in lecturer_as_list[1].lower() : title = 'DI'
    if 'dr' in lecturer_as_list[1].lower() : title = 'DR'
    if 'dr.' in lecturer_as_list[2].lower() : title += ' DR'
    return (name, rank, title)

process_lecturer_information(lecturers_raw[2][1])



('Johann Eder', 'Prof', 'DI DR')

In [21]:
lecturers = [ ( lecturer[0], *process_lecturer_information(lecturer[1]), lecturer[2], university_id ) for lecturer in lecturers_raw ]
lecturers

[('36866626',
  'Klaus Schöffmann',
  'Ass Prof',
  'DI DR',
  'ITEC',
  'ATEOS1000019137'),
 ('2077308266',
  'Dragi Kimovski',
  'Postdoc-Ass',
  'DR',
  'ITEC',
  'ATEOS1000019137'),
 ('834107405', 'Johann Eder', 'Prof', 'DI DR', 'ISYS', 'ATEOS1000019137'),
 ('772243224', 'Julius Köpke', 'Ass Prof', 'DI DR', 'ISYS', 'ATEOS1000019137')]

In [22]:
create_table_query = """
    CREATE TABLE LECTURER (
        Id VARCHAR(10) PRIMARY KEY,
        Name VARCHAR(30),
        Rank VARCHAR(20),
        Title VARCHAR(20),
        Department CHAR(4),
        University CHAR(15)
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [23]:
insert_query = """
    INSERT INTO LECTURER (Id, Name, Rank, Title, Department, University)
    VALUES (%s, %s, %s, %s, %s, %s);
"""

cursor.executemany(insert_query, lecturers)
conn.commit()

##### Creating the studyplan table

In [24]:
studyPlans = []
for level in ['bachelor_study_plans', 'master_study_plans'] :
    for studyPlan in content[level] :
        studyPlans.append( (int(studyPlan['id']), level.split('_')[0], studyPlan['branch']))
studyPlans

[(511, 'bachelor', 'Technical Studies'),
 (522, 'bachelor', 'Economoics'),
 (922, 'master', 'Economics'),
 (911, 'master', 'Technical Studies')]

In [25]:
create_table_query = """
    CREATE TABLE STUDYPLAN (
        Id INT PRIMARY KEY,
        Level VARCHAR(8),
        Branch VARCHAR(40)
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [26]:
insert_query = """
    INSERT INTO STUDYPLAN (Id, Level, Branch)
    VALUES (%s, %s, %s);
"""

cursor.executemany(insert_query, studyPlans)
conn.commit()

##### Creating the course table

In [34]:
COURSES_PATH = './aau_corses.json'

with open(COURSES_PATH, 'r') as fp:
    content = json.load(fp)
content

{'bachelor': [{'id': '620.220',
   'title': 'Einführung in die strukturierte und objektbasierte Programmierung',
   'type': 'VO',
   'lecturer': '36866626',
   'ECTS': '2',
   'department': 'ITEC'},
  {'id': '621.900',
   'title': 'Web-Technologien',
   'type': 'VO',
   'lecturer': '772243224',
   'ECTS': '2',
   'department': 'ISYS'},
  {'id': '620.050',
   'title': 'Datenbanken',
   'type': 'VO',
   'lecturer': '772243224',
   'ECTS': '2',
   'department': 'ISYS'},
  {'id': '620.050',
   'title': 'Datenbanktechnologie',
   'type': 'UE',
   'lecturer': '772243224',
   'ECTS': '2',
   'department': 'ISYS'}],
 'master': [{'id': '623.254',
   'title': 'Current Topics in Information Systems Engineering',
   'type': 'VC',
   'lecturer': '772243224',
   'department': 'AINF',
   'ECTS': '3'},
  {'id': '624.002',
   'title': 'Privatissimum für Diplomand/inn/en und Dissertant/inn/en',
   'type': 'PV',
   'lecturer': '772243224',
   'department': 'ISYS',
   'ECTS': '1'},
  {'id': '623.250',
   

In [35]:
courses = []
for level in ['bachelor', 'master'] :
    for course in content[level] :
        courses.append( (course['id'], course['type'], int(course['ECTS']), course['department'], university_name, level) )
courses = list(set(courses))
courses

[('621.900',
  'VO',
  2,
  'ISYS',
  'Alpen-Adria-Universität Klagenfurt',
  'bachelor'),
 ('623.250', 'VC', 4, 'ISYS', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('623.500', 'VC', 4, 'ISYS', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('620.050',
  'VO',
  2,
  'ISYS',
  'Alpen-Adria-Universität Klagenfurt',
  'bachelor'),
 ('624.002', 'PV', 1, 'ISYS', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('623.254', 'VC', 3, 'AINF', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('623.502', 'VC', 4, 'ITEC', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('620.220',
  'VO',
  2,
  'ITEC',
  'Alpen-Adria-Universität Klagenfurt',
  'bachelor'),
 ('623.252', 'VC', 2, 'ISYS', 'Alpen-Adria-Universität Klagenfurt', 'master'),
 ('620.050',
  'UE',
  2,
  'ISYS',
  'Alpen-Adria-Universität Klagenfurt',
  'bachelor')]

In [39]:
create_table_query = """
    CREATE TABLE COURSE (
        Id CHAR(7),
        Type CHAR(2),
        ECTS INT,
        Department CHAR(4),
        University VARCHAR(40),
        Level VARCHAR(8),
        CONSTRAINT pk PRIMARY KEY (Id, Type, Department, University)
    );
"""

cursor.execute(create_table_query)
conn.commit()

In [40]:
insert_query = """
    INSERT INTO COURSE (Id, Type, ECTS, Department, University, Level)
    VALUES (%s, %s, %s, %s, %s, %s);
"""

cursor.executemany(insert_query, courses)
conn.commit()

### Exercise 2: creating the pivot table

In [41]:
import pandas as pd

pivot_query = '''
    SELECT
        s.Name AS Student,
        l.Name AS Lecturer,
        AVG(g.Grade) AS AverageGrade
    FROM
        grade g
    JOIN
        student s ON g.StudentId = s.Id
    JOIN
        lecturer l ON g.LecturerId = l.Id
    GROUP BY
        s.Name, l.Name
    ORDER BY
        s.Name, l.Name;
    '''

cursor.execute(pivot_query)
rows = cursor.fetchall()
columns = [ desc[0] for desc in cursor.description ]
pivot_content = pd.DataFrame(rows, columns = columns)
pivot_content

,student,lecturer,averagegrade
0,Amelie Engel,Johann Eder,5.0000000000000000
1,Amelie Engel,Julius Köpke,2.0000000000000000
2,Amelie Meier,Dragi Kimovski,2.0000000000000000
3,Amelie Meier,Julius Köpke,2.5000000000000000
4,Amely Horn,Julius Köpke,4.5000000000000000
...,...,...,...
204,Tim Peters,Julius Köpke,2.5000000000000000
205,Timm Lang,Julius Köpke,3.5000000000000000
206,Timm Lang,Klaus Schöffmann,2.0000000000000000
207,Yannick Lorenz,Julius Köpke,2.5000000000000000


In [42]:
pivot = pd.pivot_table(pivot_content, values = 'averagegrade', index = 'student', columns = 'lecturer')
pivot

lecturer,Dragi Kimovski,Johann Eder,Julius Köpke,Klaus Schöffmann
student,,,,
Amelie Engel,NaN,5.0,2.0,NaN
Amelie Meier,2.0,NaN,2.5,NaN
Amely Horn,NaN,NaN,4.5,3.0
Amely Lehmann,3.0,NaN,2.0,NaN
Anna Becker,NaN,NaN,1.0,3.0
...,...,...,...,...
Sophie Brandt,NaN,NaN,2.333333,2.0
Sophie Zimmermann,NaN,NaN,3.0,2.0
Tim Peters,1.0,NaN,2.5,NaN


### Closing the connection

In [43]:
cursor.close()
conn.close()